# Objectif du script

Récupération des coordonnées GPS (et anciennement codes IRIS mais DEPRECATED car API du gouvernement n'est plus fonctionnelle)

In [90]:
import warnings
from pandas.core.common import SettingWithCopyWarning
warnings.simplefilter(action="ignore", category=SettingWithCopyWarning)

In [91]:
import pandas as pd
import numpy as np
from geopy.geocoders import Nominatim
import json
import requests
import time

In [92]:
year = '2020'
VPN_use = 1         # Si 1, on active l'utilisation de NordVPN et le fait de faire tourner l'adresse IP pour le scraping

In [93]:
import logging
logging.basicConfig(filename='./get_gps/get_gps_' + year + '.log', level=logging.INFO)

#with open('get_gps.log', 'w'):
#    pass

## Lecture du fichier Adresses

In [94]:
# Chargement du jeu de données
df = pd.read_csv('./databases/inter/01C - adresses 2016-2019.csv', sep=",",
                dtype = {'Code postal': str, 'Code departement': str, 'Code commune INSEE': str})
df.head()

,Unnamed: 0,No voie,Type de voie,Voie,Code postal,Commune,Code departement,Code commune INSEE,Ville,Adresse,year
0,2,3.0,RUE,DES CHAMPAGNES,1160,PRIAY,01,01314,1160 PRIAY,3 RUE DES CHAMPAGNES 1160 PRIAY,2019
1,3,5.0,LOT,LE BIOLAY,1370,SAINT-ETIENNE-DU-BOIS,01,01350,1370 SAINT ETIENNE DU BOIS,5 LOT LE BIOLAY 1370 SAINT-ETIENNE-DU-BOIS,2019
2,4,165.0,ALL,DES LIBELLULES,1340,ATTIGNAT,01,01024,1340 ATTIGNAT,165 ALL DES LIBELLULES 1340 ATTIGNAT,2019
3,5,9.0,RTE,DU VIADUC,1250,CIZE,01,01106,1250 CIZE,9 RTE DU VIADUC 1250 CIZE,2019
4,6,NaN,NaN,SUR LA LATIE,1250,CIZE,01,01106,1250 CIZE,SUR LA LATIE 1250 CIZE,2019


In [95]:
print(df.columns)

Index(['Unnamed: 0', 'No voie', 'Type de voie', 'Voie', 'Code postal',
       'Commune', 'Code departement', 'Code commune INSEE', 'Ville', 'Adresse',
       'year'],
      dtype='object')


In [96]:
df.shape

(3654325, 11)

## Paramètres

In [97]:
# Renseigner ici la première ligne et la dernière ligne de la base adresses qui vont être utilisées
row_first = 0
row_last = df.shape[0]

### Renseigner manuellement si on souhaite ne parser qu'une partie des adresses

row_first = 0
row_last = 1000

print("Calcul des coordonnées GPS de " + str(row_first) + " à " + str(row_last - 1))
logging.info("Calcul des coordonnées GPS de " + str(row_first) + " à " + str(row_last - 1))

Calcul des coordonnées GPS de 0 à 999


In [98]:
# Fréquence de l'export intermédiaire (si script plante à cause d'un problème d'API)
freq_export = 100

## Sélection des adresses à parser pour ajout des coordonnées GPS

In [99]:
# Sélection des adresses à partir de row_first et row_last définies plus haut
selec = df.iloc[row_first:row_last, :]
selec.shape

(1000, 11)

In [100]:
# Instanciation de la BDD de correspondance entre Code Postal / Communes et Code INSEE + GPS Communes
# La BDD se trouve ici : https://data.opendatasoft.com/explore/dataset/code-postal-code-insee-2015@public/download/?format=csv&timezone=Europe/Berlin&lang=fr&use_labels_for_header=true&csv_separator=%3B
communes = pd.read_csv('databases/code-postal-code-insee-2015.csv', sep = ";")
communes.head()

,Geo Point,Geo Shape,CODE_COM,INSEE_COM,NOM_COM,STATUT,X_CENTROID,Y_CENTROID,Z_MOYEN,SUPERFICIE,POPULATION,CODE_CANT,CODE_ARR,CODE_DEPT,NOM_DEPT,Code_postal,Nom_commune,coordonnees_gps,Unnamed: 18
0,"49.0097923668,0.696073334129","{""type"": ""Polygon"", ""coordinates"": [[[0.682900...",49,27049,BEAUMESNIL,Commune simple,531927,6881409,162,1264,562,2.0,2.0,27,EURE,27330.0,MESNIL EN OUCHE,"49.0090520551,0.696415474297",NaN
1,"49.0521199676,0.412267826158","{""type"": ""Polygon"", ""coordinates"": [[[0.409543...",547,27547,SAINT-GERMAIN-LA-CAMPAGNE,Commune simple,510734,6886673,176,2231,895,3.0,2.0,27,EURE,27230.0,ST GERMAIN LA CAMPAGNE,"49.0522921876,0.412852228157",NaN
2,"48.7896521776,0.276557382012","{""type"": ""Polygon"", ""coordinates"": [[[0.260047...",347,61347,RESENLIEU,Commune simple,500194,6858135,231,504,210,21.0,2.0,61,ORNE,61230.0,RESENLIEU,"48.7900887821,0.276930426076",NaN
3,"48.8470682746,0.643176111978","{""type"": ""Polygon"", ""coordinates"": [[[0.628763...",359,27359,JUIGNETTES,Commune simple,527172,6863726,228,1302,243,6.0,3.0,27,EURE,27250.0,JUIGNETTES,"48.8472706788,0.64345379282",NaN
4,"48.8662500091,0.559682765078","{""type"": ""Polygon"", ""coordinates"": [[[0.553791...",136,61136,COUVAINS,Commune simple,521103,6865598,236,1788,167,18.0,2.0,61,ORNE,NaN,NaN,NaN,NaN


In [101]:
# On ne garde que les colonnes qui nous intéressent 
# (On a les coordonnees gps dedans, ce sera intéressant pour faire un check des coordonnees qu'on récupère plus bas)

communes = communes[['INSEE_COM', 'NOM_COM', 'Code_postal', 'Nom_commune', 'coordonnees_gps']]
communes = communes.rename(columns = {'NOM_COM': "Commune"})
communes.head()

,INSEE_COM,Commune,Code_postal,Nom_commune,coordonnees_gps
0,27049,BEAUMESNIL,27330.0,MESNIL EN OUCHE,"49.0090520551,0.696415474297"
1,27547,SAINT-GERMAIN-LA-CAMPAGNE,27230.0,ST GERMAIN LA CAMPAGNE,"49.0522921876,0.412852228157"
2,61347,RESENLIEU,61230.0,RESENLIEU,"48.7900887821,0.276930426076"
3,27359,JUIGNETTES,27250.0,JUIGNETTES,"48.8472706788,0.64345379282"
4,61136,COUVAINS,NaN,NaN,NaN


In [102]:
# Instanciation d'un dataframe communes_ville qui ne conserve que le code INSEE, les coordonnés, et la ville
communes_ville = communes
communes_ville['Code_postal'] = communes_ville['Code_postal'].fillna(-1).astype(int).astype(str).replace('-1', np.nan)
communes_ville['Ville'] = communes_ville['Code_postal'] + " " + communes_ville['Commune']
communes_ville['Ville'] = communes_ville['Ville'].where(-communes_ville['Code_postal'].isna(), communes_ville['Commune'])
communes_ville = communes_ville.drop(columns = {'Commune', 'Code_postal', 'Nom_commune'})
communes_ville['Ville'] = communes_ville['Ville'].replace('-', ' ', regex=True).astype(str)
communes_ville.head()

,INSEE_COM,coordonnees_gps,Ville
0,27049,"49.0090520551,0.696415474297",27330 BEAUMESNIL
1,27547,"49.0522921876,0.412852228157",27230 SAINT GERMAIN LA CAMPAGNE
2,61347,"48.7900887821,0.276930426076",61230 RESENLIEU
3,27359,"48.8472706788,0.64345379282",27250 JUIGNETTES
4,61136,NaN,COUVAINS


In [103]:
# Ajout du code commune INSEE et Coordonnées GPS à la sélection d'adresses
selec = selec.merge(communes_ville, on = "Ville", how = "left")
selec = selec.drop(columns = {'Unnamed: 0'})
selec.head()

,No voie,Type de voie,Voie,Code postal,Commune,Code departement,Code commune INSEE,Ville,Adresse,year,INSEE_COM,coordonnees_gps
0,3.0,RUE,DES CHAMPAGNES,1160,PRIAY,01,01314,1160 PRIAY,3 RUE DES CHAMPAGNES 1160 PRIAY,2019,01314,"46.0197521486,5.28523353803"
1,5.0,LOT,LE BIOLAY,1370,SAINT-ETIENNE-DU-BOIS,01,01350,1370 SAINT ETIENNE DU BOIS,5 LOT LE BIOLAY 1370 SAINT-ETIENNE-DU-BOIS,2019,01350,"46.2749447261,5.28428557632"
2,165.0,ALL,DES LIBELLULES,1340,ATTIGNAT,01,01024,1340 ATTIGNAT,165 ALL DES LIBELLULES 1340 ATTIGNAT,2019,01024,"46.2861802203,5.1795233845"
3,9.0,RTE,DU VIADUC,1250,CIZE,01,01106,1250 CIZE,9 RTE DU VIADUC 1250 CIZE,2019,01106,"46.2003848172,5.45082526739"
4,NaN,NaN,SUR LA LATIE,1250,CIZE,01,01106,1250 CIZE,SUR LA LATIE 1250 CIZE,2019,01106,"46.2003848172,5.45082526739"


In [104]:
selec['Commune_Lat'] = selec['coordonnees_gps'].str.split(",", expand = True)[0]
selec['Commune_Lon'] = selec['coordonnees_gps'].str.split(",", expand = True)[1]
selec = selec.drop(columns = {'coordonnees_gps'})
selec.head()

,No voie,Type de voie,Voie,Code postal,Commune,Code departement,Code commune INSEE,Ville,Adresse,year,INSEE_COM,Commune_Lat,Commune_Lon
0,3.0,RUE,DES CHAMPAGNES,1160,PRIAY,01,01314,1160 PRIAY,3 RUE DES CHAMPAGNES 1160 PRIAY,2019,01314,46.0197521486,5.28523353803
1,5.0,LOT,LE BIOLAY,1370,SAINT-ETIENNE-DU-BOIS,01,01350,1370 SAINT ETIENNE DU BOIS,5 LOT LE BIOLAY 1370 SAINT-ETIENNE-DU-BOIS,2019,01350,46.2749447261,5.28428557632
2,165.0,ALL,DES LIBELLULES,1340,ATTIGNAT,01,01024,1340 ATTIGNAT,165 ALL DES LIBELLULES 1340 ATTIGNAT,2019,01024,46.2861802203,5.1795233845
3,9.0,RTE,DU VIADUC,1250,CIZE,01,01106,1250 CIZE,9 RTE DU VIADUC 1250 CIZE,2019,01106,46.2003848172,5.45082526739
4,NaN,NaN,SUR LA LATIE,1250,CIZE,01,01106,1250 CIZE,SUR LA LATIE 1250 CIZE,2019,01106,46.2003848172,5.45082526739


In [105]:
selec['lon'] = ""
selec['lat'] = ""
selec.head()

,No voie,Type de voie,Voie,Code postal,Commune,Code departement,Code commune INSEE,Ville,Adresse,year,INSEE_COM,Commune_Lat,Commune_Lon,lon,lat
0,3.0,RUE,DES CHAMPAGNES,1160,PRIAY,01,01314,1160 PRIAY,3 RUE DES CHAMPAGNES 1160 PRIAY,2019,01314,46.0197521486,5.28523353803,,
1,5.0,LOT,LE BIOLAY,1370,SAINT-ETIENNE-DU-BOIS,01,01350,1370 SAINT ETIENNE DU BOIS,5 LOT LE BIOLAY 1370 SAINT-ETIENNE-DU-BOIS,2019,01350,46.2749447261,5.28428557632,,
2,165.0,ALL,DES LIBELLULES,1340,ATTIGNAT,01,01024,1340 ATTIGNAT,165 ALL DES LIBELLULES 1340 ATTIGNAT,2019,01024,46.2861802203,5.1795233845,,
3,9.0,RTE,DU VIADUC,1250,CIZE,01,01106,1250 CIZE,9 RTE DU VIADUC 1250 CIZE,2019,01106,46.2003848172,5.45082526739,,
4,NaN,NaN,SUR LA LATIE,1250,CIZE,01,01106,1250 CIZE,SUR LA LATIE 1250 CIZE,2019,01106,46.2003848172,5.45082526739,,


In [106]:
selec.shape

(1000, 15)

## Récupération des coordonnées GPS à partir de l'API Nominatim

In [107]:
# printing letters
import string
import random

user_agent = ''

def change_user_agent():
    letters = string.ascii_letters
    user_agent = ''.join(random.choice(letters) for i in range(8))
    return user_agent
    
user_agent = change_user_agent()
print(user_agent)

bdrywjWU


In [108]:
# Initialisation de Nord VPN pour le scraping si l'option est activée plus haut

if VPN_use == 1:
    from nordvpn_switcher import initialize_VPN,rotate_VPN,terminate_VPN

    for i in range(5):
        try:
            settings = initialize_VPN(save = 1,area_input = ['random countries europe 10'])    # ['complete rotation'] for complete rotation
        except:   
            pass

You're using Windows.
Performing system check...
###########################

NordVPN installation check: ✓
NordVPN service check: ✓
Opening NordVPN app and disconnecting if necessary...
NordVPN app launched: ✓
#####################################

You've entered a list of connection options. Checking list...


Saving settings in project folder...


Done!

You're using Windows.
Performing system check...
###########################

NordVPN installation check: ✓
NordVPN service check: ✓
Opening NordVPN app and disconnecting if necessary...
NordVPN app launched: ✓
#####################################

You've entered a list of connection options. Checking list...

You're using Windows.
Performing system check...
###########################

NordVPN installation check: ✓
NordVPN service check: ✓
Opening NordVPN app and disconnecting if necessary...
NordVPN app launched: ✓
#####################################

You've entered a list of connection options. Checking list...


Saving settin

In [109]:
#while True: 
#    rotate_VPN(settings)
#    time.sleep(3600) #e.g. rotate servers every hour

In [110]:
from time import sleep

# Get GPS Coordinates from addresses

geolocator = Nominatim(user_agent = user_agent)

start = time.time()
etape = time.time()
print("Heure début : ")
print(time.strftime('%H:%M:%S', time.gmtime(start)))
print("\n")

logging.info("Heure début : " + time.strftime('%H:%M:%S', time.gmtime(start)) + "\n")

n = 0

for row in range(0, row_last - row_first):
    
    longitude_api = ""
    latitude_api = ""
    longitude_adresse = ""
    latitude_adresse = ""
   
    adresse = selec['Adresse'][row]
    ville = selec['Ville'][row]
    code_INSEE = selec['Code commune INSEE'][row]
    
    
    
    # Interrogation de l'API Nominatim : récupération des coordonnées GPS de l'adresse
    
    try:
        location = geolocator.geocode(adresse, country_codes = 'fr', timeout = 100)
    
    except requests.exceptions.RequestException as e:
        latitude_api = ""
        longitude_api = ""
        raise SystemExit(e)
        logging.info(adresse + ": " + e)
        continue
        
    if location is None:    # Sinon aucun résultat, on ne met rien dans les coordonnées GPS qui vont servir à interroger les codes IRIS
        latitude_api = ""
        longitude_api = ""
    
    else:
        latitude_api = location.latitude
        longitude_api = location.longitude
                   
    print(adresse + " ===> " + str(latitude_api) + "," + str(longitude_api))
       
    selec['lon'][row] = longitude_api
    selec['lat'][row] = latitude_api
    
#    # Récupération des codes IRIS (codes quartiers) !!! DEPRECATED - API n'est plus disponible
#    if isinstance(code_INSEE, float):     # Si pas de code commune
#        print(adresse + ': pas de code commune INSEE')
#        logging.info(adresse + ': pas de code commune INSEE')
#        iris = ""
#    
#    elif latitude_api == "":               # Si pas de coordonnées GPS
#        print(adresse + ": pas de coordonnees GPS pour code IRIS")
#        logging.info(adresse + ": pas de coordonnees GPS pour code IRIS")
#        iris = ""
#    
#    else:                              # Sinon on interroge l'API du gouv pour récupérer le code IRIS
#        try:
#            result = requests.get('https://geo.api.gouv.fr/iris?lon=' + str(longitude_api) + '&lat=' 
#                             + str(latitude_api) + '&codeCommune=' + str(code_INSEE), timeout = 5)
#        except requests.exceptions.RequestException as e: 
#            logging.info(adresse + ": " + e)
#            raise SystemExit(e)
#
#        if result.ok == True:
#            obj = json.loads(result.text)
#            iris = obj['codeIris']
#            #print(obj['codeIris'])
#        else:
#            result.close()
#
#        print(adresse + " ===> " + str(latitude_api) + "," + str(longitude_api) + " (code IRIS = " + str(iris) + ")")
#    
#    selec['code_iris'][row] = iris

    n = n + 1
    
    # Toutes les 100 requêtes : on calcule le temps des 100 dernières requêtes
    
    if (n % 100 == 0):
        print("\n")
        print("Etape : " + str(n))
        print(time.strftime('%H:%M:%S', time.gmtime(time.time())))
        print("Temps pour 100 : " + str(time.time() - etape) + " sec")
        print("\n")
        logging.info("Etape : " + str(n) + ", Temps pour 100 items: " + str(time.time() - etape) + " sec")
        etape = time.time()

    # Toutes les 1000 requêtes : on change de VPN
    
    if (n % 1000 == 0) & (VPN_use == 1):
        try:
            rotate_VPN(settings)
        except:
            pass

    # Toutes les "freq_export" requêtes, on exporte les résultats et on change de User Agent

    if (n % freq_export == 0):
        selec.iloc[(n - freq_export) : n, :].to_csv(
            './get_gps/export_adresses_inter_' + str(row_first + n - freq_export) + "-" + str(row_first + n - 1) + '_' + year + '.csv')
        logging.info("02 - Export intermédiaire : " + str(row_first + n - freq_export) + "-" + str(row_first + n - 1))
        
        change_user_agent()
        geolocator = Nominatim(user_agent = user_agent)

        
end = time.time()
print("\n")
print("Heure fin : ")
print(time.strftime('%H:%M:%S', time.gmtime(end)))
print("\n")

logging.info("Heure fin : " + time.strftime('%H:%M:%S', time.gmtime(end)) + "\n")
logging.info("Temps de calcul : " + str(end - start) + ' sec')

print("Temps de calcul : ")
print(str(round(((end - start)/60), 2)) + ' min')

Heure début : 
15:20:15


3 RUE DES CHAMPAGNES 1160 PRIAY ===> 46.0019301,5.2876139
5 LOT LE BIOLAY 1370 SAINT-ETIENNE-DU-BOIS ===> 46.2934726,5.3001737
165 ALL DES LIBELLULES 1340 ATTIGNAT ===> 46.2690381,5.1745553
9 RTE DU VIADUC 1250 CIZE ===> 46.205148,5.452365
SUR LA LATIE 1250 CIZE ===> ,
50 RUE DOC NODET 1000 BOURG-EN-BRESSE ===> ,
EN CHEVRET 1250 TOSSIAT ===> ,
1724 RTE DE MONTLEGER 1560 MANTENAY-MONTLIN ===> 46.438633,5.118969
1250 RTE DE CUET 1340 MONTREVEL-EN-BRESSE ===> 46.321708,5.112519
181 RTE DE VERNOUX 1560 COURTES ===> 46.468146,5.103709
CROCU 1560 SAINT-TRIVIER-DE-COURTES ===> 46.4628038,5.0427754
LE PIOCHAY 1560 SAINT-TRIVIER-DE-COURTES ===> ,
GRANDVAL 1560 SAINT-TRIVIER-DE-COURTES ===> 46.4606544,5.0557583
FARNOZ 1560 SAINT-TRIVIER-DE-COURTES ===> ,
5575  MOLARDOURY 1560 SAINT-TRIVIER-DE-COURTES ===> 46.4568175,5.0653117
MONTDIDIER 1310 MONTRACOL ===> ,
544 RTE DE BOIS BRULE 1310 MONTRACOL ===> ,
5362  LA MORANDIERE 1240 CERTINES ===> 46.1439607,5.2658321
51 RUE CH

242 RTE DE PONT DE VAUX 1190 SERMOYER ===> 46.491278,4.978009
PRES BLANCS 1310 CONFRANCON ===> ,
CHAMP GUILLEMAUD 1310 CONFRANCON ===> 46.2774041,5.0792858
28 RUE DE FRANCHE COMTE 1250 NIVIGNE ET SURAN ===> 46.2670214,5.4265621
LA NEUVE OUEST 1440 VIRIAT ===> ,
10 RUE BRICHEMER 1000 BOURG-EN-BRESSE ===> 46.2127046,5.2522479
CHALANDRE 1000 SAINT-DENIS-LES-BOURG ===> 46.2086788,5.1894808
4 IMP DES HIRONDELLES 1190 PONT-DE-VAUX ===> 46.43283,4.94236
5072  AUX CHARMES 1190 GORREVOD ===> 46.420515,4.9377549
6 RUE LOUIS BLERIOT 1000 BOURG-EN-BRESSE ===> 46.2116055,5.2356948
LES BLANCHETS 1310 BUELLAS ===> ,
5386  LA POTIERE 1340 MONTREVEL-EN-BRESSE ===> 46.3087372,5.0991369
LA POTIERE 1340 MONTREVEL-EN-BRESSE ===> 46.3087372,5.0991369
49 ALL DE LA GARE 1340 ATTIGNAT ===> 46.287456,5.1584449
471 RUE DES ADAMS 1340 CRAS-SUR-REYSSOUZE ===> 46.3130788,5.1631944
LES ADAMS 1340 CRAS-SUR-REYSSOUZE ===> 46.3134688,5.1637043
8 ALL DE L'AULNAIE 1440 VIRIAT ===> ,
5502  LES MALADIERES 1851 MARBOZ ===> 

11 RUE HENRI DUNANT 1000 BOURG-EN-BRESSE ===> 46.1804664,5.3208671
461 RUE DU DOCTEUR HUBERT 1160 NEUVILLE-SUR-AIN ===> 46.0782372,5.376654
23 RUE GEN DELESTRAINT 1000 BOURG-EN-BRESSE ===> ,
16 RUE LAMARTINE 1000 BOURG-EN-BRESSE ===> 46.2028099,5.2041821
237 RUE BRICHEMER 1000 BOURG-EN-BRESSE ===> 46.213313,5.251875
2 RUE DU PALAIS 1000 BOURG-EN-BRESSE ===> 46.2047895,5.2239094
LES RODETS 1270 VILLEMOTIER ===> 46.335371,5.3170981
395 RTE DE STRASBOURG 1270 VILLEMOTIER ===> 46.3569673,5.3290775
450 CHE DE LA BETRANE 1370 SAINT-ETIENNE-DU-BOIS ===> 46.2943884,5.2826692
BOIS RENAUDATS 1370 COURMANGOUX ===> ,
PRE ROUGE 1370 MEILLONNAS ===> 46.2478993,5.3472546
38 RUE YVES MONTAND 1000 BOURG-EN-BRESSE ===> 46.2078127,5.2471801
MONT CURTET 1250 JASSERON ===> 46.2146621,5.3451843
SOUS L'ORME 1250 JASSERON ===> ,
SOUS LE MOULIN 1250 JASSERON ===> ,
186 RTE DES BEYS 1570 FEILLENS ===> 46.3425271,4.8830519
LES GOUILLES 1570 MANZIAT ===> ,
LA TIENNAZ 1570 MANZIAT ===> ,
LES BLOTTONS 1570 MANZIAT 

53 RUE LOUISE DE SAVOIE 1160 PONT-D AIN ===> 46.050662,5.3437344
LES POCHONS 1340 BRESSE VALLONS ===> 46.3115031,5.1669766
672 RTE DU COTTELARD 1560 LESCHEROUX ===> 46.399917,5.138017
10 RTE DU TRAM 1290 CORMORANCHE-SUR-SAONE ===> 46.2394508,4.8367963
LE GAVILLON 1290 CORMORANCHE-SUR-SAONE ===> ,
CLAVIERE 1290 GRIEGES ===> ,
5156  A MONTAPLAN 1290 LAIZ ===> 46.2533314,4.8892662
5189  LE METRILLOT 1250 MONTAGNAT ===> 46.1661201,5.2719625
317 RUE DES VAVRES 1000 SAINT-DENIS-LES-BOURG ===> 46.2033482,5.205967
107 CHE DES LAZARISTES 1000 SAINT-DENIS-LES-BOURG ===> 46.2056195,5.2075724
308 RUE LOUIS FRANCOIS JAMBON 1250 TOSSIAT ===> 46.1413315,5.316481
5196  OUSSIAT 1160 PONT-D AIN ===> 46.0557947,5.3611957
5382 RUE SAINT EXUPERY 1160 PONT-D AIN ===> 46.052454,5.3351751
5468  RONGEON 1340 JAYAT ===> 46.3475849,5.1226298
15 RUE DE MONTHOLON 1000 BOURG-EN-BRESSE ===> 46.2002537,5.2105831
47 CHE DES BOULEAUX 1960 PERONNAS ===> 46.1792811,5.2224576
LE SABLON 1560 SAINT-JULIEN-SUR-REYSSOUZE ===>

LE CANTON 1250 JASSERON ===> 46.2317904,5.2917693
1 RUE SAINT EXUPERY 1160 PONT-D AIN ===> 46.0493333,5.3358711
COTES DU PORT 1160 PONT-D AIN ===> ,
13 ALL DES PINSONS 1000 BOURG-EN-BRESSE ===> 46.184619,5.245856
5737  DERRIERE LE CHATEAU 1160 PONT-D AIN ===> ,
93 GR GRANDE RUE 1290 PONT-DE-VEYLE ===> ,
ETANG BLOTONNE 1190 SAINT-BENIGNE ===> ,
262 RTE DE CHAZELLES 1340 MARSONNAS ===> 46.338605,5.069247
PRE LUC 1340 MARSONNAS ===> 46.3391886,5.0648516
5823  EN DOMAGNE 1250 CEYZERIAT ===> ,
21 RUE DU PORT 1160 NEUVILLE-SUR-AIN ===> 46.0777024,5.3769728
7 ALL VINCENT BENONY 1000 BOURG-EN-BRESSE ===> 46.2018551,5.2369916
AU MAS BALLET 1250 TOSSIAT ===> ,
195 CHEM DE MAS BALLET 1250 TOSSIAT ===> ,
5291  LES TEPPES 1380 SAINT-CYR-SUR-MENTHON ===> 46.276249,4.9555716
321 RTE DE NEUVILLE 1250 VILLEREVERSURE ===> 46.1894883,5.3972703
162 ALL DES AVIATEURS 1000 BOURG-EN-BRESSE ===> 46.1894365,5.236975
1 RUE GUICHARD 1000 BOURG-EN-BRESSE ===> 46.2053183,5.2263779
235 CHE DE L'ANCIENNE TUILERIE 19

107 CHE DU PELOUX 1310 POLLIAT ===> 46.2391432,5.1661091
LA RAVETTE 1310 POLLIAT ===> 46.2385658,5.1660807
6703  LE BUIDON 1440 VIRIAT ===> 46.2327209,5.2301886
80 RUE DE LA CABORNE 1380 SAINT-ANDRE-DE-BAGE ===> 46.2975777,4.9146958
2239 RTE DE COUR 1380 BAGE-DOMMARTIN ===> 46.335953,4.94395
SUR LA VIGNE GUYON 1250 POUILLAT ===> ,
EN COMBESSORET 1250 POUILLAT ===> ,
DEVANT L'EGLISE 1250 POUILLAT ===> ,
CHAMPS FLEURIS 1250 POUILLAT ===> ,
VERGERS BOURGEONS 1250 POUILLAT ===> ,
LE MOULINET 1250 POUILLAT ===> ,
LE BOURBOUILLON 1250 POUILLAT ===> ,
A LA COURBE 1250 POUILLAT ===> ,
7 RUE CESAR 1000 BOURG-EN-BRESSE ===> 46.208402,5.222233
GRANDS PRES DES GUILLEMOTS 1190 CHAVANNES-SUR-REYSSOUZE ===> ,
3771 RTE DE MANTENAY 1190 CHAVANNES-SUR-REYSSOUZE ===> 46.4286562,5.0255217
LES GUILLEMOTS 1190 CHAVANNES-SUR-REYSSOUZE ===> 46.4291028,5.0248308
LE BRET 1851 MARBOZ ===> 46.3342387,5.201008
603 RUE DU PUITS GUILLEMIN 1750 REPLONGES ===> 46.2970108,4.8756693
4 RUE ALFRED BERTHOLET 1000 BOURG-EN-

FONTAINE COLLON 1190 REYSSOUZE ===> ,
70 RUE DU THIOUDET 1960 PERONNAS ===> 46.1700212,5.1929663
MONTERNOZ 1960 PERONNAS ===> 46.1663721,5.2015486
51 RUE DU 1ER SEPTEMBRE 1944 1160 PONT-D AIN ===> 46.0492395,5.3345726
LES TEPPES 1160 PONT-D AIN ===> ,
20 AV DES GRANGES BARDES 1000 BOURG-EN-BRESSE ===> 46.2196427,5.2467334


Etape : 900
15:27:56
Temps pour 100 : 52.21890640258789 sec


6 RUE MALHERBE 1000 BOURG-EN-BRESSE ===> 46.190949,5.229197
RUE MALHERBE 1000 BOURG-EN-BRESSE ===> 46.1914553,5.228892
11 RUE DOC NODET 1000 BOURG-EN-BRESSE ===> ,
526 RTE DES PERTHUISETTES 1340 BRESSE VALLONS ===> 46.3092504,5.1729515
80 AV DE MACON 1000 BOURG-EN-BRESSE ===> 46.2112591,5.2184654
BD EDOUARD HERRIOT 1000 BOURG-EN-BRESSE ===> 46.20982,5.225767
5272  LA ROVARY 1250 MONTAGNAT ===> ,
5062  LES GAUTHIERS 1960 SERVAS ===> 46.1484561,5.1892389
LES GAUTHIERS 1960 SERVAS ===> 46.1484561,5.1892389
LE MANILLER 1370 BENY ===> 46.3159821,5.2880019
5154  LE MANILLER 1370 BENY ===> 46.3159821,5.2880019
2

In [111]:
if VPN_use == 1:
    terminate_VPN(instructions=None)

Trying to load saved settings...
Saved settings loaded!


Disconnecting...
Done!


In [112]:
selec.head()

,No voie,Type de voie,Voie,Code postal,Commune,Code departement,Code commune INSEE,Ville,Adresse,year,INSEE_COM,Commune_Lat,Commune_Lon,lon,lat
0,3.0,RUE,DES CHAMPAGNES,1160,PRIAY,01,01314,1160 PRIAY,3 RUE DES CHAMPAGNES 1160 PRIAY,2019,01314,46.0197521486,5.28523353803,5.287614,46.00193
1,5.0,LOT,LE BIOLAY,1370,SAINT-ETIENNE-DU-BOIS,01,01350,1370 SAINT ETIENNE DU BOIS,5 LOT LE BIOLAY 1370 SAINT-ETIENNE-DU-BOIS,2019,01350,46.2749447261,5.28428557632,5.300174,46.293473
2,165.0,ALL,DES LIBELLULES,1340,ATTIGNAT,01,01024,1340 ATTIGNAT,165 ALL DES LIBELLULES 1340 ATTIGNAT,2019,01024,46.2861802203,5.1795233845,5.174555,46.269038
3,9.0,RTE,DU VIADUC,1250,CIZE,01,01106,1250 CIZE,9 RTE DU VIADUC 1250 CIZE,2019,01106,46.2003848172,5.45082526739,5.452365,46.205148
4,NaN,NaN,SUR LA LATIE,1250,CIZE,01,01106,1250 CIZE,SUR LA LATIE 1250 CIZE,2019,01106,46.2003848172,5.45082526739,,


In [113]:
selec.shape

(1000, 15)

## Export de la totalité de la sélection d'adresses après la fin du job

In [ ]:
selec.to_csv('./export_gps/02 - export_adresses_' + str(row_first) + "-" + str(row_last - 1) + '_' + year + '.csv', index = False)